In [ ]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from keras import backend as K
import csv
import math as math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from keras.layers.merge import concatenate as concat 
from keras.models import  Model, load_model
from keras.layers import  LeakyReLU 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model, to_categorical
from keras.losses import mse, BinaryCrossentropy
import scipy.stats

from sklearn.model_selection import train_test_split
disable_eager_execution()

In [ ]:
batch_size = 256
no_epochs = 400
latent_dim = 4 # Dimension of the latent space
nt=10 # Size of the latent space vectors (first VAE)
beta = 10 # Value beta for the loss function


In [ ]:
filename = '../input/data-controlvae/latent.csv' # Directory containing the latent space coordinates for the first type of trajectories
raw_data = open(filename, 'rt')
reader = csv.reader(raw_data, delimiter=',')
x = list(reader)
datalat = np.array(x).astype('float')
print(datalat.shape)


In [ ]:
filenamet = '../input/data-controlvae/latentb.csv' # Directory containing the latent space coordinates for the second type of trajectories
raw_datat = open(filenamet, 'rt')
readert = csv.reader(raw_datat, delimiter=',')
t = list(readert)
datalatb = np.array(t).astype('float')
print(datalatb.shape)


In [ ]:
filenamez = '../input/data-controlvae/score.csv' # Directory containing the scores corresponding to the latent space coordinates for the first type of trajectories
raw_dataz = open(filenamez, 'rt')
readerz = csv.reader(raw_dataz, delimiter=',')
zc = list(readerz)
datasc = np.array(zc).astype('float')
print(datasc.shape)
o=0

for i in range (0,120000):
    r= datasc[i]
    if r >=0.0 and r<=1.0:
        o=o+1 
        #print(o)
        #print(r)
labels = np.zeros((o))
lat= np.zeros((o,10))
y=0
for i in range (0,120000):
    r=datasc[i]
    if r>=0.0 and r<=1.0:
        labels[y]=r
        lat[y][:]=datalat[i][:]
        y=y+1
     
print(labels)
print(lat[0][:])

In [ ]:
filenamez = '../input/data-controlvae/scoreb.csv' # Directory containing the scores corresponding to the latent space coordinates for the second type of trajectories
raw_dataz = open(filenamez, 'rt')
readerz = csv.reader(raw_dataz, delimiter=',')
zc = list(readerz)
datascb = np.array(zc).astype('float')
print(datascb.shape)
o=0

# Balancing the datasets and introducing the labels
for i in range (0,120000):
    r= datascb[i]
    if r >=0.0 and r<=1.0:
        o=o+1 
        #print(o)
        #print(r)
labelbs = np.zeros((o))
latb= np.zeros((o,10))
y=0
for i in range (0,120000):
    r=datascb[i]
    if r>=0.0 and r<=1.0:
        labelbs[y]=r
        latb[y][:]=datalatb[i][:]
        y=y+1


print(labelbs[0])
print(latb[0][:])
a = len(labels)+len(labelbs)
b = len(lat)+len(latb)
print(b)
l= np.zeros((a))
la=np.zeros((a,10))
y=0
for i in range(0,len(labels)):
    l[i]=labels[i]
for i in range(len(labels),a):
    l[i]=labelbs[y]
    y=y+1
y=0
for i in range(0,len(labels)):
    la[i]=lat[i]
for i in range(len(labels),a):
    la[i][:]=latb[y][:]
    y=y+1
print(a)
print(len(l))
print(len(la))

print(l[0])
print(la[0][:])

for i in range (0,len(l)):
    r=l[i]
    if r <=0.15 and r>=0.0 :
        l[i]=0
    if r <=0.25 and r>0.15 :
        l[i]=1
    if r <=0.35 and r>0.25 :
        l[i]=2
    if r <=0.45 and r>0.35 :
        l[i]=3
    if r <=0.55 and r>0.45 :
        l[i]=4
    if r <=0.65 and r>0.55 :
        l[i]=5
    if r <=0.75 and r>0.65 :
        l[i]=6
    if r <=0.85 and r>0.75 :
        l[i]=7
    if r <=0.95 and r>0.85 :
        l[i]=8
    if r <=1.0 and r>0.95 :
        l[i]=9

print(l[1000])
target=np.zeros((len(l),2))
for i in range (0,len(l)):
    p = la[i][8]
    if p == 0:
        target[i][0]=0
        target[i][1]=1
    else:
        target[i][0]=1
        target[i][1]=0
    

In [ ]:
# Formation of random validation, training, and test dataset
x_temp, x_test, y_temp, y_test, t_temp, t_test = train_test_split(la, l,target, test_size = 0.01)

x_train, x_valid, y_train, y_valid, t_train, t_valid = train_test_split(x_temp,  y_temp, t_temp, test_size=0.1)

n_train = len(x_train)
n_test = len(x_test)

print(n_train)
print(n_test)
print(y_train.shape)
print(y_train[1])
print(x_train.shape)


In [ ]:
# One-hot encoding labels
Y_train= to_categorical(y_train,10)
Y_test= to_categorical(y_test,10)
Y_valid= to_categorical(y_valid,10)
print(Y_train.shape)
ny = Y_train.shape[1]
print(Y_train[193])

In [ ]:
# Encoder's input layer
Xi = Input(shape=(nt,))
label = Input(shape=(ny,))
tar = Input(shape=(2,))
inputs = concat([Xi, label, tar])

In [ ]:
# Definition
encoded = Dense(256, activation = 'tanh')(inputs)
encoded = Dense(128, activation = 'tanh')(encoded)
encoded = Dense(64, activation = 'tanh')(encoded)
encoded = Dense(32, activation = 'tanh')(encoded)
mu      = Dense(latent_dim, name='latent_m')(encoded)
sigma   = Dense(latent_dim, name='latent_sigma')(encoded)

In [ ]:
# reparameterization trick
def sample_z(args):
  mu, sigma = args
  batch     = K.shape(mu)[0]
  dim       = K.int_shape(mu)[1]
  eps       = K.random_normal(shape=(batch, dim))
  return mu + K.exp(sigma / 2) * eps

z       = Lambda(sample_z, output_shape=(latent_dim, ), name='z')([mu, sigma])
zc = concat([z,label,tar])

In [ ]:
# Instantiate encoder
encoder = Model([Xi,label,tar], [mu, sigma, zc], name='encoder')
encoder.summary()

In [ ]:
# Definition Decoder
d_i   = Input((latent_dim+ny+2, ))
decoded = Dense(32, activation='tanh')(d_i)
decoded = Dense (64, activation = 'tanh')(decoded)
decoded = Dense (128, activation = 'tanh')(decoded)
decoded = Dense (256, activation = 'tanh')(decoded)
oa     = Dense(nt, name='decoder_outputa')(decoded)


In [ ]:
# Decoder
decoder = Model(d_i, [oa], name='decoder')
decoder.summary()

In [ ]:
#  VAE
vae_outputs = decoder(encoder([Xi,label,tar])[2])
vae = Model([Xi,label,tar], vae_outputs, name='vae')
vae.summary()

In [ ]:
# Define loss
def kl_reconstruction_loss(i, o):
  # Reconstruction loss
  reconstruction_loss = mse(K.flatten(i),K.flatten(o))
  # KL divergence loss
  kl_loss = beta*(1 + sigma - K.square(mu) - K.exp(sigma))
  kl_loss = K.sum(kl_loss, axis=-1)
  kl_loss *= -0.5
  # Total loss = 50% rec + 50% KL divergence loss
  return K.mean(reconstruction_loss + kl_loss)

In [ ]:
# Compile VAE
vae.compile(optimizer='adam', loss=kl_reconstruction_loss, experimental_run_tf_function=False)

# Train autoencoder
results = vae.fit([x_train,Y_train, t_train], [x_train], epochs = no_epochs, batch_size = batch_size, validation_data = ([x_valid,Y_valid, t_valid], [x_valid]))

In [ ]:
# QC training and validation curves (should follow eachother)
plt.figure(figsize=(8,2))
plt.plot(results.history['val_loss'], label='val')
plt.plot(results.history['loss'], label='train')
plt.xlabel('epoch index')
plt.ylabel('loss value ')
plt.legend()
plt.show()

In [ ]:
# Test
encoded_test = encoder.predict([x_test,Y_test, t_test])
vae_test = vae.predict([x_test,Y_test, t_test])


print(x_test[28])
print(vae_test[28])

In [ ]:
#Generate new latent space coordinates

import random
def construct(digit,digitb, z = None):
  out = np.zeros((1,latent_dim+ny+2))
 
  out[0,0]=random.uniform(-10,10)
  out[0,1]=random.uniform(-10,10)
  out[0,2]=random.uniform(-10,10)
  out[0,3]=random.uniform(-10,10)
  

  out[:,digitb+latent_dim]=1.
  out[:,digit+latent_dim+ny]=1.
  if z is None:
    return (out)
  else:
    for i in range (len(z)):
      out[:,i]=z[i]
    return out

In [ ]:
#Generation of new latent space coordinates corresponding to the desires legibility level.

lat = np.zeros((300,10))
latb = np.zeros((300,10))

for i in range (0,300):
  
    if i>= 0 and i<30:
        sample = construct(0,0)
        sampleb = construct(1,0)
    if i>= 30 and i<60:
        sample = construct(0,1)
        sampleb = construct(1,1)
    if i>= 60 and i<90:
        sample = construct(0,2)
        sampleb = construct(1,2)
    if i>= 90 and i<120:
        sample = construct(0,3)
        sampleb = construct(1,3)
    if i>= 120 and i<150:
        sample = construct(0,4)
        sampleb = construct(1,4)
    if i>= 150 and i<180:
        sample = construct(0,5)
        sampleb = construct(1,5)
    if i>= 180 and i<210:
        sample = construct(0,6)
        sampleb = construct(1,6)
    if i>= 210 and i<240:
        sample = construct(0,7)
        sampleb = construct(1,7)
    if i>= 240 and i<270:
        sample = construct(0,8)
        sampleb = construct(1,8)
    if i>= 270 and i<300:
        sample = construct(0,9)
        sampleb = construct(1,9)

    print(sample)
    s=decoder.predict(sample)
    sb=decoder.predict(sampleb)
    print(s)
    lat[i,:]=s
    latb[i,:]=sb

print(lat[0][5])    

filelat = open ('control.csv','w+', newline='')
with filelat:
    write = csv.writer(filelat)
    write.writerows(lat)
filelatb = open ('controlb.csv','w+', newline='')
with filelatb:
    write = csv.writer(filelatb)
    write.writerows(latb)